<a href="https://colab.research.google.com/github/anubhavshrestha/sift_feature_extraction/blob/main/CIFAR_10_FIN_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import cv2
import numpy as np
from sklearn.decomposition import PCA

In [ ]:
# Define PCA-based feature extraction
def extract_sift_and_reduce_dimensionality(image, pca):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    if descriptors is not None:
        reduced_descriptors = pca.transform(descriptors)
        return reduced_descriptors
    else:
        return None

# Perform PCA dimensionality reduction
def reduce_dimensionality(descriptors, n_components=64):
    pca = PCA(n_components=n_components)
    reduced_descriptors = pca.fit_transform(descriptors)
    return pca, reduced_descriptors

# Initialize PCA and perform dimensionality reduction (you may adjust n_components)
pca, _ = reduce_dimensionality(train_sift_features, n_components=64)

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import cv2
import numpy as np

# Define a function to extract SIFT features from an image
def extract_sift_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return descriptors

# Define a custom dataset that computes SIFT features for each image
class SIFTDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        sift_features = extract_sift_features(img)  # Extract SIFT features
        if self.transform:
            img = self.transform(img)
        return sift_features, label

# Create data transforms (you can modify these as needed)
transforms_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=None)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=None)

# Create custom SIFT datasets
sift_trainset = SIFTDataset(trainset, transform=transforms_train)
sift_testset = SIFTDataset(testset, transform=transforms_test)

# Define your neural network model (modify as needed)
class SimpleFNN(nn.Module):
    def __init__(self):
        super(SimpleFNN, self).__init__()
        self.fc1 = nn.Linear(128, 64)  # Input size 128 (SIFT feature dimension), output size 64
        self.fc2 = nn.Linear(64, 10)   # Output size 10 (number of classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model and optimizer
model = SimpleFNN()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Set device to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# DataLoader for SIFT datasets
batch_size = 128
sift_trainloader = torch.utils.data.DataLoader(sift_trainset, batch_size=batch_size, shuffle=True)
sift_testloader = torch.utils.data.DataLoader(sift_testset, batch_size=batch_size, shuffle=False)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    total_correct = 0
    total_samples = 0

    for sift_features, labels in sift_trainloader:
        sift_features, labels = sift_features.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(sift_features.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(sift_trainloader):.4f}, "
          f"Accuracy: {100 * total_correct / total_samples:.2f}%")

print("Training finished!")

# Evaluate the model on the test set
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for sift_features, labels in sift_testloader:
        sift_features, labels = sift_features.to(device), labels.to(device)

        outputs = model(sift_features.float())
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
